## 복습

RNN의 한계

기울기 소실, 기울기 폭주, 긴 학습시간이라는 '장기 의존성 문제'가 존재한다.

LSTM의 한계

input gate, output gate, forget gate를 운용하기 때문에, 계산이 복잡하고 시간이 오래 걸린다. 또한, 긴 문장이라면 기울기 소실 문제가 발생한다.

Seq2Seq 모델은, input 내용을 다른 시퀀스로 변환하는 모델 구조를 갖고 있고, 입력과 출력의 단어 개수가 같지 않아도 되고, 형태 및 내용에 있어 변환이 필요한 경우 모두 적용할 수 있다.

그런데, 기존 seq2seq 모델의 경우, 인코더가 input sequence의 모든 정보를 압축하여 고정된 크기의 context vector 하나를 만드는 구조였고, 디코더는 이 context vector 만을 참고하여 출력 시퀀스를 생성한다.

따라서, 고정된 크기의 context vector를 사용하여 입력 문장이 길어지면 모든 정보를 고정된 크기에 압축하는 것이 어려워 일부 데이터의 소실이 발생하게 되었다.

### Attention의 등장

위의 고정된 크기의 context vector에 긴 문장을 넣게 되어 일부 데이터가 소실되는 문제를 해결하기 위해 '가장 관련 있는 정보'만을 활용해 새로운 context vector를 생성하게 되었다.

## Attention

attention: 가장 중요한 부분에 더 높은 가중치를 부여하여 참고하는 것이고, 즉 출력 단어를 만들 때 입력 문장에서 어떤 단어에 집중할 것인지를 고르는 기술이다.

### Attention의 과정
1. 입력을 벡터 형태로 변화하여 각 요소를 수치화
2. 각 요소 별 중요도 점수 산출
3. 중요도에 따른 입력 정보 조합

### Attention의 장점
1. 시간 경과에 따른 유연성 - 모든 단어에서 다른 모든 단어로 직접적인 연결고리를 형성하여 멀리 떨어진 단어 사이의 관계도 정보 손실 없이 명확히 파악이 가능하다.
2. 공간에 대한 유연성 - 픽셀 간의 거리에 상관없이 한 번에 전체 이미지의 전역적 관계를 학습할 수 있다.
3. 병렬화 - 문장 내 모든 단어의 관계를 한번에, 독립적으로 계산할 수 있다.

## Self-Attention

### Query, Key, Value

1. Query: '질문' 또는 '요청'으로 내가 알고 싶거나, 초점을 맞추는 대상
2. Key: 모든 정보들이 달고 있는 '이름표' 혹은 '색인'
3. Value: 키와 한쌍으로 묶여 있는 '실제 내용물'

### 기존 cross-attention vs. Self-attention

1. 기존 cross-attention
- 하나의 시퀀스가 완전히 다른 시퀀스를 참고하여 정보를 만들어내는 메커니즘
- 서로 다른 시퀀스를 참조
- 인코더와 디코더를 연결하고 정보의 병목 현상을 해소하기 위한 목적
- RNN (LSTM, GRU) 기반
- 병렬 처리 불가능

2. self-attention
- 한 문장 내 모든 단어끼리 서로의 관계를 계산
- 동일한 시퀀스를 참조
- 시퀀스 내부의 문맥 및 의존 관계를 파악하기 위한 목적
- 트랜스포머 기반
- 병렬 처리 가능

### Dot-Product Attention

attention score을 계산하는 방법 중 하나로, 쿼리와 키 벡터를 내적하여 유사도를 구한다.
1. 쿼리벡터와 모든 키 벡터를 각각 내적하여 유사도 계산을 진행
2. 계산된 값에 softmax 함수를 적용하여 종합이 1인 attention weight를 산출
3. 이 가중치를 밸류 벡터에 곱하여 최종 결과를 계산

-> 그러나 벡터의 차원이 커질수록 내적 결과가 너무 커지거나 작아지는 문제가 발생하여 기울기 소실문제가 발생한다.

### Scaled Dot-Product Attnetion

1. 쿼리벡터와 모든 키 벡터를 각각 내적하여 유사도 계산
2. 계산된 값을 스케일링
3. 스케일링을 진행한 값에 softmax 함수를 적용하여 종합이 1인 attention weight를 산출
4. 가중치를 밸류 벡터에 곱하여 최종 결과값을 계산

## Multi-Head Attention

### single-head attention

문장 내 단어들 간의 관계를 파악하기 위해 하나의 가중치 행렬만을 학습하고 번역에 사용한다.

작동 방식
1. 512차원의 입력 벡터
2. 이 벡터를 위한 어텐션 가중치 분포 계산
3. 가중합을 통해 새로운 512차원의 벡터를 출력

한계
- 하나의 어텐션 분포 안에서 각 단어 간의 문법, 의미, 위치 관계 등의 모든 종류의 정보를 한 번에 가중 평균을 구하여 각 단어 간의 중요한 관계를 놓칠 수 있다.

### Multi-Head Attention

1. 분할 - 각 헤드에 단어 벡터를 할당
2. 병렬 어텐션 계산 - 각 헤드들이 각자 분석을 수행
3. 결합 및 최종 투영 - 분석 결과 취합

## Transformer

### 전처리 단계

1. 토큰화 - 입력 텍스트를 모델이 처리할 수 있는 단위로 나누는 것
2. 임베딩 - 토큰화된 각 단위 숫자의 벡터로 변환, 즉 단어의 의미를 공간적으로 표현하는 것이며, 의미적으로 유사하면 유사한 숫자로 변환된다.
3. positional encoding - 단어들의 순서 정보를 임베딩 벡터에 추가하는 단계

### 인코더와 디코더

1. 인코더

트랜스포머의 인코더는 입력 문장을 이해하고 요약된 의미 벡터로 변환하며, 전처리과정과 인코더 레이어로 구성되어 있다.

-> 인코더 레이어?

인코더 레이어는 multi-head attention과 feed-forward로 구성되어 있으며, 이때 feed-forward는 입력 벡터의 차원을 확장하고 비선형 변환을 적용하여 새로운 표현을 생성하는 신경망 구조이다.

-> 잔차 연결?

입력 값을 그대로 출력에 더해 주는 것으로 layer가 많이 쌓여도 학습이 잘된다.

-> 층 정규화?

입력 벡터의 평균과 분산을 사용해서 정규화하는 것으로 안정화하고 빠르게 모델이 수렴하도록 한다.

2. 디코더

트랜스포머의 디코더는 인코더가 분석한 입력 문장의 의미 벡터를 받아서 출력 문장을 순차적으로 생성하며, 전처리과정, 디코더레이어, 선형레이어와 소프트맥스 레이어로 구성되어 있다.

-> 디코더 레이어?

디코더 레이어는 masked multi-head attention, encoder-decoder multi-head attention, feed-forward layer로 구성되어 있다.

-> masked multi-head attention?

미래 시점의 단어 정보를 참고하지 못하도록 마스크를 적용하는 어텐션

-> encoder-decoder attention?

디코더가 인코더의 출력을 참고하면서 현재 생성 중인 단어를 입력 문장의 의미와 연결하는 과정


## 전체 모델 및 데이터 흐름

1. 입력 준비: 문장을 [의미+위치] 벡터로 바꾸기
2. 인코더: 입력 문장의 [문맥적 의미] 깊이를 이해하기
3. 디코더: 번역 문장을 [한 단어씩] 생성하기